In [1]:
import os.path
import os
import sys
from matplotlib import rcParams
from matplotlib.animation import ArtistAnimation
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
#import ctables
import matplotlib.colors as colors

from datetime import datetime, timedelta
from siphon.cdmr import Dataset
import numpy as np
import numpy.ma as ma
import netCDF4
import math

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature

projection = ccrs.LambertConformal(central_longitude=262.5, 
                                   central_latitude=38.5, 
                                   standard_parallels=(38.5, 38.5),
                                    globe=ccrs.Globe(semimajor_axis=6371229,semiminor_axis=6371229))

from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS
import scipy.ndimage as ndimage
import scipy.signal as signal

import metpy
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import sky_cover
from metpy.calc import (bunkers_storm_motion, bulk_shear, dewpoint, dewpoint_from_relative_humidity, dry_lapse, moist_lapse,
                        vapor_pressure, saturation_vapor_pressure, wind_speed, wind_direction, pressure_to_height_std,
                        mixing_ratio, cape_cin, wind_components, height_to_pressure_std, equivalent_potential_temperature,
                        parcel_profile, precipitable_water, storm_relative_helicity, mean_pressure_weighted, 
                        most_unstable_cape_cin, most_unstable_parcel, supercell_composite, significant_tornado, get_layer,
                        relative_humidity_from_dewpoint, surface_based_cape_cin, mixed_layer_cape_cin,
                        surface_based_cape_cin, potential_temperature, wind_direction, add_pressure_to_height,
                        add_height_to_pressure, divergence, vorticity, lat_lon_grid_deltas, mixed_parcel,
                        most_unstable_parcel, lcl, lfc, mixing_ratio_from_relative_humidity, el,  cape_cin,
                        height_to_pressure_std, dewpoint_from_relative_humidity, dewpoint_from_specific_humidity)
from metpy.units import units
from metpy.plots import SkewT, Hodograph
from metpy.interpolate import interpolate_1d as metinterp, log_interpolate_1d
from metpy.calc.tools import get_layer, get_layer_heights
from metpy.plots.declarative import BarbPlot, ContourPlot, FilledContourPlot, MapPanel, PanelContainer, PlotObs

import sharppy
import sharppy.sharptab.profile as profile
import sharppy.sharptab.interp as interp
import sharppy.sharptab.winds as winds
import sharppy.sharptab.utils as utils
import sharppy.sharptab.params as params
import sharppy.sharptab.thermo as thermo

import pickle
import csv
import pandas as pd
import xarray as xr
import seaborn as sns
from scipy import stats
import s3fs

import time
from ecape.calc import calc_ecape

In [2]:
##########################################################
# Pre Load HRRR Files to speed up parameter calculations #
##########################################################
year=2019
month=5

fs = s3fs.S3FileSystem(anon=True)

times523 = [1]
d2={}
for n in range(0,len(times523)):
    day=23
    hour=times523[n]
    time_start = datetime(int(year), int(month), int(day), int(hour), 0) # Our specified time
    month = time_start.month
    if month < 10:
        month = '0'+str(month)
    if hour < 10:
        hour = '0'+str(hour)
        
    timeIN=str(str(year)+str(month)+str(day))
    files = np.array(fs.ls('s3://noaa-hrrr-bdp-pds/hrrr.'+str(timeIN)+'/conus/'))
    fileN = np.where(files=='noaa-hrrr-bdp-pds/hrrr.'+str(timeIN)+'/conus/hrrr.t'+str(hour)+'z.wrfnatf00.grib2')
    fs.get(files[fileN[0][0]], files[fileN[0][0]].split('/')[-1])
    fname=files[fileN[0][0]].split('/')[-1]
    d2["c{0}".format(n)] = xr.open_dataset(fname, filter_by_keys={'typeOfLevel': 'hybrid'})
    
#print(d1['c0'].attrs)

In [ ]:
hour = 1

if hour==1:
    c=d2['c0']      
else:
    print("WRONG HOUR")

time_start = datetime(int(year), int(month), int(day), int(hour), 0) # Our specified time
hour = time_start.hour
if hour < 10:
    hour = '0'+str(hour)
day = time_start.day
if day < 10:
    day = '0'+str(day)
month = time_start.month
if month < 10:
    month = '0'+str(month)

timeIN=str(str(year)+str(month)+str(day))
##########################################################
BulkShear_0_0_1_km_BWD = []
BulkShear_1_0_3_km_BWD = []
BulkShear_2_0_5_km_BWD = []
BulkShear_3_0_6_km_BWD = []
BulkShear_4_0_8_km_BWD = []

Buoyancy_10_ECAPE = []

Composite_10_ml01ehi = []
Composite_11_ml03ehi = []
Composite_12_mu01ehi = []
Composite_13_mu03ehi = []

Inhibition_3_ESL25 = []
Inhibition_4_ESL40 = []

SRFlow_4_0_6kmSRflow = []
SRFlow_5_0_8kmSRflow = []
SRFlow_7_4_6kmSRflow = []
#########################################################
lev=np.asarray(c.variables['hybrid'][:]) # 0 = lowest level, 49 = highest level
lat=np.asarray(c.variables['latitude'])
lon=((np.asarray(c.variables['longitude'])*-1)+360)*-1

T=np.asarray(c.variables['t'][:]) #K #temperature
q=np.asarray(c.variables['q'][:]) # kg kg**-1 #specific humidity
uwnd=np.asarray(c.variables['u'][:]) #m/s #u wind
vwnd=np.asarray(c.variables['v'][:]) #m/s #v wind
hgt=np.asarray(c.variables['gh'][:]) #geopotential meters #geopotential height
VVEL_up=(np.asarray(c.variables['w'][:])) #Pa s**-1 #vertical velocity  
lev=(np.asarray(c.variables['pres'][:])/100.) #hPa #pressure
Td=dewpoint_from_specific_humidity(lev* units('hPa'),T* units('kelvin'),q)

NElat, NElon = 33.0, -93.0
NWlat, NWlon = 33.0, -101.0
SElat, SElon = 38.0, -93.0
SWlat, SWlon = 38.0, -101.0

dlonNE = np.abs(lon - NElon)
dlatNE = np.abs(lat - NElat)
cordmaxNE = np.maximum(dlonNE,dlatNE)
xNE,yNE=np.where(cordmaxNE == np.min(cordmaxNE))
xNE=xNE[0]
yNE=yNE[0]

dlonNW = np.abs(lon - NWlon)
dlatNW = np.abs(lat - NWlat)
cordmaxNW = np.maximum(dlonNW,dlatNW)
xNW,yNW=np.where(cordmaxNW == np.min(cordmaxNW))
xNW=xNW[0]
yNW=yNW[0]

dlonSE = np.abs(lon - SElon)
dlatSE = np.abs(lat - SElat)
cordmaxSE = np.maximum(dlonSE,dlatSE)
xSE,ySE=np.where(cordmaxSE == np.min(cordmaxSE))
xSE=xSE[0]
ySE=ySE[0]

dlonSW = np.abs(lon - SWlon)
dlatSW = np.abs(lat - SWlat)
cordmaxSW = np.maximum(dlonSW,dlatSW)
xSW,ySW=np.where(cordmaxSW == np.min(cordmaxSW))
xSW=xSW[0]
ySW=ySW[0]

xNE = 327
xSE = 513
ySE = 1040
ySW = 790

for i in range (327,513):
    for j in range (790,1040):
        startT = time.perf_counter()
        P_sounding = lev[:,i,j]* units.hectopascal
        T_sounding = (T[:,i,j]* units.kelvin).to(units.degC)
        Td_sounding = Td[:,i,j].to(units.degC)
        Td_sounding[np.isnan(Td_sounding)] = 0*units('degC')
        hgt_sounding = (hgt[:,i,j] - hgt[0,i,j])* units.meter
        msl_sounding = (hgt[:,i,j])* units.meter
        u_sounding = (uwnd[:,i,j]* units.meter_per_second).to(units.knot)
        v_sounding = (vwnd[:,i,j]* units.meter_per_second).to(units.knot)
        Omega=VVEL_up[:,i,j]*(units.pascal/units.second)
        RH_skewt = relative_humidity_from_dewpoint(T_sounding, Td_sounding)
        
        prof = profile.create_profile(profile='default', pres=P_sounding, hght=hgt_sounding, tmpc=T_sounding, dwpc=Td_sounding, u=u_sounding, v=v_sounding, missing=-9999)
        mupcl = params.parcelx( prof, flag=3) # Most-Unstable Parcel
        mlpcl = params.parcelx( prof, flag=4)

        Q_skewt = mixing_ratio_from_relative_humidity(P_sounding, T_sounding, RH_skewt)
        
        ml_0_1ehi = sharppy.sharptab.params.ehi(prof, mlpcl, hbot= 0, htop = 1000) 
        ml_0_3ehi = sharppy.sharptab.params.ehi(prof, mlpcl, hbot= 0, htop = 3000)
        mu_0_1ehi = sharppy.sharptab.params.ehi(prof, mupcl, hbot= 0, htop = 1000)
        mu_0_3ehi = sharppy.sharptab.params.ehi(prof, mupcl, hbot= 0, htop = 3000)
        
        sfc = prof.pres[prof.sfc]
        p1km = interp.pres(prof, interp.to_msl(prof, 1000.))
        p3km = interp.pres(prof, interp.to_msl(prof, 3000.))
        p4km = interp.pres(prof, interp.to_msl(prof, 4000.))
        p5km = interp.pres(prof, interp.to_msl(prof, 5000.))
        p6km = interp.pres(prof, interp.to_msl(prof, 6000.))
        p8km = interp.pres(prof, interp.to_msl(prof, 8000.))
        
        sfc_1km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p1km)
        sfc1shear = utils.mag(sfc_1km_shear[0], sfc_1km_shear[1])
        sfc_3km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p3km)
        sfc3shear = utils.mag(sfc_3km_shear[0], sfc_3km_shear[1])
        sfc_8km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p8km)
        sfc8shear = utils.mag(sfc_8km_shear[0], sfc_8km_shear[1])      
        sfc_6km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p6km)
        sfc6shear = utils.mag(sfc_6km_shear[0], sfc_6km_shear[1])
        sfc_5km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p5km)
        sfc5shear = utils.mag(sfc_5km_shear[0], sfc_5km_shear[1] )
        
        srwind = params.bunkers_storm_motion(prof)
        
        srflow06km_vector = winds.sr_wind(prof, pbot = sfc, ptop = p6km, stu = srwind[0], stv = srwind[1])
        srflow08km_vector = winds.sr_wind(prof, pbot = sfc, ptop = p8km, stu = srwind[0], stv = srwind[1])
        srflow4_6km_vector = winds.sr_wind(prof, pbot = p4km, ptop = p6km, stu = srwind[0], stv = srwind[1])

        srflow06km = utils.mag(srflow06km_vector[0], srflow06km_vector[1])
        srflow08km = utils.mag(srflow08km_vector[0], srflow08km_vector[1])
        srflow4_6km = utils.mag(srflow4_6km_vector[0], srflow4_6km_vector[1])       
        
        ecape = calc_ecape(msl_sounding, P_sounding, T_sounding, Q_skewt, u_sounding, v_sounding)

        presArray=np.array([890.,880.,870.,860.,850.,840.,830.,820.,810.,800.,790.,780.,770.,760.,750.,740.,730.,720.,710.,700.,
                       690.,680.,670.,660.,650.,640.,630.,620.,610.,600.])

        lapse1 = sharppy.sharptab.params.lapse_rate(prof, lower = 900., upper =890.)
        lapse2 = sharppy.sharptab.params.lapse_rate(prof, lower = 890., upper =880.)
        lapse3 = sharppy.sharptab.params.lapse_rate(prof, lower = 880., upper =870.)
        lapse4 = sharppy.sharptab.params.lapse_rate(prof, lower = 870., upper =860.)
        lapse5 = sharppy.sharptab.params.lapse_rate(prof, lower = 860., upper =850.)
        lapse6 = sharppy.sharptab.params.lapse_rate(prof, lower = 850., upper =840.)
        lapse7 = sharppy.sharptab.params.lapse_rate(prof, lower = 840., upper =830.)
        lapse8 = sharppy.sharptab.params.lapse_rate(prof, lower = 830., upper =820.)
        lapse9 = sharppy.sharptab.params.lapse_rate(prof, lower = 820., upper =810.)
        lapse10 = sharppy.sharptab.params.lapse_rate(prof, lower = 810., upper =800.)
        lapse11 = sharppy.sharptab.params.lapse_rate(prof, lower = 800., upper =790.)
        lapse12 = sharppy.sharptab.params.lapse_rate(prof, lower = 790., upper =780.)
        lapse13 = sharppy.sharptab.params.lapse_rate(prof, lower = 780., upper =770.)
        lapse14 = sharppy.sharptab.params.lapse_rate(prof, lower = 770., upper =760.)
        lapse15 = sharppy.sharptab.params.lapse_rate(prof, lower = 760., upper =750.)
        lapse16 = sharppy.sharptab.params.lapse_rate(prof, lower = 750., upper =740.)
        lapse17 = sharppy.sharptab.params.lapse_rate(prof, lower = 740., upper =730.)
        lapse18 = sharppy.sharptab.params.lapse_rate(prof, lower = 730., upper =720.)
        lapse19 = sharppy.sharptab.params.lapse_rate(prof, lower = 720., upper =710.)
        lapse20 = sharppy.sharptab.params.lapse_rate(prof, lower = 710., upper =700.)
        lapse21 = sharppy.sharptab.params.lapse_rate(prof, lower = 700., upper =690.)
        lapse22 = sharppy.sharptab.params.lapse_rate(prof, lower = 690., upper =680.)
        lapse23 = sharppy.sharptab.params.lapse_rate(prof, lower = 680., upper =670.)
        lapse24 = sharppy.sharptab.params.lapse_rate(prof, lower = 670., upper =660.)
        lapse25 = sharppy.sharptab.params.lapse_rate(prof, lower = 660., upper =650.)
        lapse26 = sharppy.sharptab.params.lapse_rate(prof, lower = 650., upper =640.)
        lapse27 = sharppy.sharptab.params.lapse_rate(prof, lower = 640., upper =630.)
        lapse28 = sharppy.sharptab.params.lapse_rate(prof, lower = 630., upper =620.)
        lapse29 = sharppy.sharptab.params.lapse_rate(prof, lower = 620., upper =610.)
        lapse30 = sharppy.sharptab.params.lapse_rate(prof, lower = 610., upper =600.)

        hght900 = interp.hght(prof, 900.)
        hght890 = interp.hght(prof, 890.)
        hght880 = interp.hght(prof, 880.)
        hght870 = interp.hght(prof, 870.)
        hght860 = interp.hght(prof, 860.)
        hght850 = interp.hght(prof, 850.)
        hght840 = interp.hght(prof, 840.)
        hght830 = interp.hght(prof, 830.)
        hght820 = interp.hght(prof, 820.)
        hght810 = interp.hght(prof, 810.)
        hght800 = interp.hght(prof, 800.)
        hght790 = interp.hght(prof, 790.)
        hght780 = interp.hght(prof, 780.)
        hght770 = interp.hght(prof, 770.)
        hght760 = interp.hght(prof, 760.)
        hght750 = interp.hght(prof, 750.)
        hght740 = interp.hght(prof, 740.)
        hght730 = interp.hght(prof, 730.)
        hght720 = interp.hght(prof, 720.)
        hght710 = interp.hght(prof, 710.)
        hght700 = interp.hght(prof, 700.)
        hght690 = interp.hght(prof, 690.)
        hght680 = interp.hght(prof, 680.)
        hght670 = interp.hght(prof, 670.)
        hght660 = interp.hght(prof, 660.)
        hght650 = interp.hght(prof, 650.)
        hght640 = interp.hght(prof, 640.)
        hght630 = interp.hght(prof, 630.)
        hght620 = interp.hght(prof, 620.)
        hght610 = interp.hght(prof, 610.)
        hght600 = interp.hght(prof, 600.)

        lapseArray=np.array([lapse1,lapse2,lapse3,lapse4,lapse5,lapse6,lapse7,lapse8,lapse9,lapse10,
                            lapse11,lapse12,lapse13,lapse14,lapse15,lapse16,lapse17,lapse18,lapse19,lapse20,
                            lapse21,lapse22,lapse23,lapse24,lapse25,lapse26,lapse27,lapse28,lapse29,lapse30])

        hghtArray=np.array([hght890,hght880,hght870,hght860,hght850,hght840,hght830,hght820,hght810,hght800,
                           hght790,hght780,hght770,hght760,hght750,hght740,hght730,hght720,hght710,hght700,
                           hght690,hght680,hght670,hght660,hght650,hght640,hght630,hght620,hght610,hght600,])

        ESL25 = np.asarray(np.where(lapseArray < 2.5))
        if len(ESL25[0]) != 0:
            ESL25hgtArray = hghtArray[ESL25[0][0]:ESL25[0][-1]]
            ESL25depth = hghtArray[ESL25[0][-1]] - hghtArray[ESL25[0][0]]
        else:
            ESL25depth = 0

        ESL40 = np.asarray(np.where(lapseArray < 4.0))
        if len(ESL40[0]) != 0:
            ESL40hgtArray = hghtArray[ESL40[0][0]:ESL40[0][-1]]
            ESL40depth = hghtArray[ESL40[0][-1]] - hghtArray[ESL40[0][0]]
        else:
            ESL40depth = 0        
        ###################################################################################       
        BulkShear_0_0_1_km_BWD.append(sfc1shear)
        BulkShear_1_0_3_km_BWD.append(sfc3shear)
        BulkShear_2_0_5_km_BWD.append(sfc5shear)
        BulkShear_3_0_6_km_BWD.append(sfc6shear)
        BulkShear_4_0_8_km_BWD.append(sfc8shear)

        Buoyancy_10_ECAPE.append(ecape.magnitude)

        Composite_10_ml01ehi.append(ml_0_1ehi)
        Composite_11_ml03ehi.append(ml_0_3ehi)
        Composite_12_mu01ehi.append(mu_0_1ehi)
        Composite_13_mu03ehi.append(mu_0_3ehi)

        Inhibition_3_ESL25.append(ESL25depth)
        Inhibition_4_ESL40.append(ESL40depth)

        SRFlow_4_0_6kmSRflow.append(srflow06km)
        SRFlow_5_0_8kmSRflow.append(srflow08km)
        SRFlow_7_4_6kmSRflow.append(srflow4_6km)
        
        endT = time.perf_counter()
        print(i,j, "Elapsed (after compilation) = {}s".format((endT - startT)))
        j+=1
    #notify.send('column'+ str(i) +' completed')
    i+=1

BulkShear_0_0_1_km_BWD = np.asarray(BulkShear_0_0_1_km_BWD)
BulkShear_1_0_3_km_BWD = np.asarray(BulkShear_1_0_3_km_BWD)
BulkShear_2_0_5_km_BWD = np.asarray(BulkShear_2_0_5_km_BWD)
BulkShear_3_0_6_km_BWD = np.asarray(BulkShear_3_0_6_km_BWD)
BulkShear_4_0_8_km_BWD = np.asarray(BulkShear_4_0_8_km_BWD)

Buoyancy_10_ECAPE = np.asarray(Buoyancy_10_ECAPE)

Composite_10_ml01ehi = np.asarray(Composite_10_ml01ehi)
Composite_11_ml03ehi = np.asarray(Composite_11_ml03ehi)
Composite_12_mu01ehi = np.asarray(Composite_12_mu01ehi)
Composite_13_mu03ehi = np.asarray(Composite_13_mu03ehi)

Inhibition_3_ESL25 = np.asarray(Inhibition_3_ESL25)
Inhibition_4_ESL40 = np.asarray(Inhibition_4_ESL40)

SRFlow_4_0_6kmSRflow = np.asarray(SRFlow_4_0_6kmSRflow)
SRFlow_5_0_8kmSRflow = np.asarray(SRFlow_5_0_8kmSRflow)
SRFlow_7_4_6kmSRflow = np.asarray(SRFlow_7_4_6kmSRflow)

BulkShear_0_0_1_km_BWD = np.array_split(BulkShear_0_0_1_km_BWD, 186)
BulkShear_1_0_3_km_BWD = np.array_split(BulkShear_1_0_3_km_BWD, 186)
BulkShear_2_0_5_km_BWD = np.array_split(BulkShear_2_0_5_km_BWD, 186)
BulkShear_3_0_6_km_BWD = np.array_split(BulkShear_3_0_6_km_BWD, 186)
BulkShear_4_0_8_km_BWD = np.array_split(BulkShear_4_0_8_km_BWD, 186)

Buoyancy_10_ECAPE = np.array_split(Buoyancy_10_ECAPE, 186)

Composite_10_ml01ehi = np.array_split(Composite_10_ml01ehi, 186)
Composite_11_ml03ehi = np.array_split(Composite_11_ml03ehi, 186)
Composite_12_mu01ehi = np.array_split(Composite_12_mu01ehi, 186)
Composite_13_mu03ehi = np.array_split(Composite_13_mu03ehi, 186)

Inhibition_3_ESL25 = np.array_split(Inhibition_3_ESL25, 186)
Inhibition_4_ESL40 = np.array_split(Inhibition_4_ESL40, 186)

SRFlow_4_0_6kmSRflow = np.array_split(SRFlow_4_0_6kmSRflow, 186)
SRFlow_5_0_8kmSRflow = np.array_split(SRFlow_5_0_8kmSRflow, 186)
SRFlow_7_4_6kmSRflow = np.array_split(SRFlow_7_4_6kmSRflow, 186)

hr01_BulkShear_0_0_1_km_BWD = np.asarray(BulkShear_0_0_1_km_BWD)
hr01_BulkShear_1_0_3_km_BWD = np.asarray(BulkShear_1_0_3_km_BWD)
hr01_BulkShear_2_0_5_km_BWD = np.asarray(BulkShear_2_0_5_km_BWD)
hr01_BulkShear_3_0_6_km_BWD = np.asarray(BulkShear_3_0_6_km_BWD)
hr01_BulkShear_4_0_8_km_BWD = np.asarray(BulkShear_4_0_8_km_BWD)

hr01_Buoyancy_10_ECAPE = np.asarray(Buoyancy_10_ECAPE)

hr01_Composite_10_ml01ehi = np.asarray(Composite_10_ml01ehi)
hr01_Composite_11_ml03ehi = np.asarray(Composite_11_ml03ehi)
hr01_Composite_12_mu01ehi = np.asarray(Composite_12_mu01ehi)
hr01_Composite_13_mu03ehi = np.asarray(Composite_13_mu03ehi)

hr01_Inhibition_3_ESL25 = np.asarray(Inhibition_3_ESL25)
hr01_Inhibition_4_ESL40 = np.asarray(Inhibition_4_ESL40)

hr01_SRFlow_4_0_6kmSRflow = np.asarray(SRFlow_4_0_6kmSRflow)
hr01_SRFlow_5_0_8kmSRflow = np.asarray(SRFlow_5_0_8kmSRflow)
hr01_SRFlow_7_4_6kmSRflow = np.asarray(SRFlow_7_4_6kmSRflow)

np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_BulkShear_0_0_1_km_BWD.csv",hr01_BulkShear_0_0_1_km_BWD,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_BulkShear_1_0_3_km_BWD.csv",hr01_BulkShear_1_0_3_km_BWD,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_BulkShear_2_0_5_km_BWD.csv",hr01_BulkShear_2_0_5_km_BWD,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_BulkShear_3_0_6_km_BWD.csv",hr01_BulkShear_3_0_6_km_BWD,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_BulkShear_4_0_8_km_BWD.csv",hr01_BulkShear_4_0_8_km_BWD,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_Buoyancy_10_ECAPE.csv",hr01_Buoyancy_10_ECAPE,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_Composite_10_ml01ehi.csv",hr01_Composite_10_ml01ehi,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_Composite_11_ml03ehi.csv",hr01_Composite_11_ml03ehi,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_Composite_12_mu01ehi.csv",hr01_Composite_12_mu01ehi,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_Composite_13_mu03ehi.csv",hr01_Composite_13_mu03ehi,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_Inhibition_3_ESL25.csv",hr01_Inhibition_3_ESL25,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_Inhibition_4_ESL40.csv",hr01_Inhibition_4_ESL40,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_SRFlow_4_0_6kmSRflow.csv",hr01_SRFlow_4_0_6kmSRflow,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_SRFlow_5_0_8kmSRflow.csv",hr01_SRFlow_5_0_8kmSRflow,delimiter=",",fmt='%s')
np.savetxt("C:/Users/KyleD/Desktop/HRRROuts/Hourly_Array_1/hr01_SRFlow_7_4_6kmSRflow.csv",hr01_SRFlow_7_4_6kmSRflow,delimiter=",",fmt='%s')